# 读取相机内参
```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
# 将pyorbbecsdk的动态链接库所在的文件夹，添加到Python Path
import sys
import os

# 导入阿凯写的Orbbec工具库, 确保orbbec_utils.py跟你目前所执行的脚本在同一级目录下
from orbbecsdk_utils import *
# 添加Python Path
add_path_pyorbbecsdk()

# 导入pyorbbecsdk
from pyorbbecsdk import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

## 相机初始化

In [3]:
# 连接设备
ret, device = connect_device()
print(f"[INFO] 设备是否连接成功: {ret}")

# pipeline初始化
pipeline = init_pipeline(device)

[06/24 13:54:03.365455][info][19190][Context.cpp:68] Context created with config: default config!
[06/24 13:54:03.365468][info][19190][Context.cpp:73] Work directory=/home/kyle/Project/奥比中光3D相机/Gemini335-软件资料-阿凯爱玩机器人/02.奥比中光-pyobbecsdk示例代码(Gemini335), SDK version=v1.10.5-20240511-7dc8be9
[06/24 13:54:03.365479][info][19190][LinuxPal.cpp:31] createObPal: create LinuxPal!
[06/24 13:54:03.433024][info][19190][LinuxPal.cpp:142] Create PollingDeviceWatcher!
[06/24 13:54:03.433057][info][19190][DeviceManager.cpp:15] Current found device(s): (1)
[06/24 13:54:03.433061][info][19190][DeviceManager.cpp:24] 	- Name: Orbbec Gemini 335, PID: 0x0800, SN/ID: CP1L44P00049, Connection: USB3.2
[INFO] 设备是否连接成功: True
[06/24 13:54:03.902880][error][19190][HostProtocol.cpp:420] Set property value failed! propertyId:63, rc:{statusCode: 9, respErrorCode: 2, msg: }
[06/24 13:54:03.903210][error][19190][HostProtocol.cpp:420] Set property value failed! propertyId:63, rc:{statusCode: 9, respErrorCode: 2, msg: }


## 获取相机内参
**注意事项**
1. 彩色相机与红外相机有着两套独立的相机内参。需要注意的是分辨率不同，对应的相机内参也是不同的。
2. 一般我们是将深度图对齐到彩色相机坐标系下去使用的， 因此只关系彩色相机的内参参数。

In [4]:
camera_param = pipeline.get_camera_param()
print("彩色相机内参")
print(camera_param.rgb_intrinsic)
print(f"彩色相机畸变系数")
print(camera_param.rgb_distortion)
print(f"深度图相机内参")
print(camera_param.depth_intrinsic)
print(f"深度图相机畸变系数")
print(camera_param.depth_distortion)
print("深度相机到彩色相机坐标系的空间变换")
print(camera_param.transform)

彩色相机内参
<OBCameraIntrinsic fx=687.010010 fy=687.106445 cx=647.172485 cy=360.134766 width=1280 height=720>
彩色相机畸变系数
<OBCameraDistortion k1=0.005327 k2=-0.039434 k3=0.027466 k4=0.000000 k5=0.000000 k6=0.000000 p1=-0.000107 p2=-0.000340>
深度图相机内参
<OBCameraIntrinsic fx=687.010010 fy=687.106445 cx=647.172485 cy=360.134766 width=1280 height=720>
深度图相机畸变系数
<OBCameraDistortion k1=0.000000 k2=0.000000 k3=0.000000 k4=0.000000 k5=0.000000 k6=0.000000 p1=0.000000 p2=0.000000>
深度相机到彩色相机坐标系的空间变换
<OBD2CTransform rot=[1, 0, 0, 0, 1, 0, 0, 0, 1]
transform=[0, 0, 0]


In [5]:
# 获取彩色相机内参
fx = camera_param.rgb_intrinsic.fx
fy = camera_param.rgb_intrinsic.fy
cx = camera_param.rgb_intrinsic.cx
cy = camera_param.rgb_intrinsic.cy
# 彩图尺寸
img_width = camera_param.rgb_intrinsic.width
img_height = camera_param.rgb_intrinsic.height
# 构造内参矩阵
intrinsic = np.float32([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]])
print("彩色相机的内参矩阵:")
print(intrinsic)

彩色相机的内参矩阵:
[[687.01      0.      647.1725 ]
 [  0.      687.10645 360.13477]
 [  0.        0.        1.     ]]
